In [57]:
import pandas as pd
import networkx as nx
import os

In [29]:
data_N1 = pd.read_html('../data/N1.traffic.htm')
data_N1[4]

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),...,List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link ),List of links of N1 ( what is a link )
1,Link no,Name,Start location,Start location,Start location,End location,End location,End location,Length,Traffic Data,...,Traffic Data,Traffic Data,Traffic Data,Traffic Data,Traffic Data,Traffic Data,Total,Total,Total,Traffic
2,NaN,NaN,LRP,Offset,Chainage,LRP,Offset,Chainage,(Km),Heavy Truck,...,Car,Auto Rickshaw,Motor Cycle,Bi-Cycle,Cycle Rickshaw,Cart,Motorized,Non Motorized,Total AADT,(AADT)
3,N1-1L,Jatrabari - Int.with Z1101 (Left) (Left),LRPS,0,0,LRPS,822,0.822,0.822,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
4,N1-1R,Jatrabari - Int.with Z1101 (Left) (Right),LRPS,0,0,LRPS,822,0.822,0.822,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,N1-65,Coxsbazar Link Road N110-Maricha Z1009,LRP386,724,381.481,LRP403,217,398.205,16.724,24.0,...,142.0,2476.0,419.0,97.0,1042.0,0.0,4681.0,1139.0,5820.0,5820.0
104,N1-66,Maricha Z1009-Ukhia Dakbanglo Z1503,LRP403,217,398.205,LRP413,300,408.434,10.229,24.0,...,142.0,2476.0,419.0,97.0,1042.0,0.0,4681.0,1139.0,5820.0,5820.0
105,N1-67,Ukhia Dakbanglo Z1503 - Gundum Z1504,LRP413,300,408.434,LRP420,900,416.044,7.610,24.0,...,142.0,2476.0,419.0,97.0,1042.0,0.0,4681.0,1139.0,5820.0,5820.0
106,N1-68,GundumGundum Z1504 - Whykong Z1133,LRP420,900,416.044,LRP433,521,428.908,12.864,24.0,...,142.0,2476.0,419.0,97.0,1042.0,0.0,4681.0,1139.0,5820.0,5820.0


In [84]:
folder_path = '../data/Traffic_data'

dataframes = {}

for file in os.listdir(folder_path):
    if file.endswith('.traffic.htm'):
        file_path = os.path.join(folder_path, file)
        file_name = file.split('.')[0]
        df_list = pd.read_html(file_path)
        df = df_list[4]
        df = df.iloc[1:].reset_index(drop=True)
        df_key = file_name
        dataframes[df_key] = df

In [67]:
df = dataframes.get('Z8915')

def create_graph(df):
    for index, row in df.iterrows():
    # Define the start and end node for each edge.
        start_node = row['Start location']
        end_node = row['End location']
        
        # Extract traffic data to use as edge attributes.
        edge_attributes = {
            'Heavy Truck': row['Heavy Truck'],
            'Medium Truck': row['Medium Truck'],
            # Add all vehicle types you're interested in...
            'Total AADT': row['Total AADT']
        }
        
        # Add the edge to the graph.
        G.add_edge(start_node, end_node, **edge_attributes)
    
    return G

create_graph(df)

KeyError: 'Start location'

In [102]:
df = dataframes.get('Z8915')
df.iloc[1:5,:]

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
1,NaN,NaN,LRP,Offset,Chainage,LRP,Offset,Chainage,(Km),Heavy Truck,...,Car,Auto Rickshaw,Motor Cycle,Bi-Cycle,Cycle Rickshaw,Cart,Motorized,Non Motorized,Total AADT,(AADT)
2,Z8915-1,Bagmara - Bangla Bazar (Int.with R890),LRPS,0,0,LRP004,1020,4.35,4.350,0.0,...,126.0,940.0,662.0,602.0,546.0,0.0,2374.0,1148.0,3522.0,3522.0
3,Z8915-2,Bangla Bazar (Int.with R890) - Int.with Z8916,LRP004,1020,4.35,LRP010,1150,10.47,6.120,0.0,...,25.0,1025.0,872.0,396.0,380.0,0.0,2341.0,776.0,3117.0,3117.0
4,Z8915-3,Int.with Z8916 - Daulatkhan,LRP010,1150,10.47,LRP014,2290,15.66,5.190,0.0,...,25.0,1025.0,872.0,396.0,380.0,0.0,2341.0,776.0,3117.0,3117.0
